In [ ]:
# ------------------------------------ Imports ----------------------------------
from diffusers import FluxPipeline
from google.colab import userdata
from huggingface_hub import login
from IPython.display import display
from PIL import Image
import torch



# ------------------------------------ Configure API Keys / Tokens ----------------------------------
# Retrieve stored API keys from Colab's secure userdata store
openai_api_key = userdata.get('OPENAI_API_KEY')
anthropic_api_key = userdata.get('ANTHROPIC_API_KEY')
google_api_key = userdata.get('GOOGLE_API_KEY')
hf_token = userdata.get('HF_TOKEN')


# ------------------------------------ Print if Configuration was Successful ----------------------------------
print("API Keys:")
if openai_api_key:
    print(f"OpenAI API Key exists and begins {openai_api_key[:10]}")
else:
    print("OpenAI API Key not set")

if anthropic_api_key:
    print(f"Anthropic API Key exists and begins {anthropic_api_key[:10]}")
else:
    print("Anthropic API Key not set")
if google_api_key:
    print(f"Google API Key exists and begins {google_api_key[:10]}")
else:
    print("Google API Key not set")
if hf_token:
    print(f"Hugging Face Token exists and begins {hf_token[:10]}")
else:
  print("Hugging Face Token not set")


# ------------------------------------ Get GPU Info ----------------------------------
# print("\nGPU Info:")
# gpu_info = !nvidia-smi
# gpu_info = '\n'.join(gpu_info)
# if gpu_info.find('failed') >= 0:
#   print('Not connected to a GPU')
# else:
#   print(gpu_info)


# ------------------------------------ Connect to Hugging Face ----------------------------------
login(hf_token, add_to_git_credential=True)

# Request Access to HuggingFace Model:
# https://huggingface.co/black-forest-labs/FLUX.1-schnell

# Change Google Colab runtime to, 'T4 GPU'

# Load the Flux image generation pipeline from Hugging Face (a diffusion-based text-to-image generator).
# Use float16 to reduce memory usage
pipe = FluxPipeline.from_pretrained("black-forest-labs/FLUX.1-schnell",
                                    torch_dtype=torch.float16).to("cuda")

# Moves the model to GPU (cuda) for faster computation.
generator = torch.Generator(device="cuda").manual_seed(0)

# Create Prompt & Generate Image
# prompt = "A futuristic class full of students learning AI coding in the surreal style of Salvador Dali"

prompt = """
Generate an image in a surrealist style featuring majestic tigers roaming a vast,
ethereal landscape suspended high above a shimmering ocean of clouds. The ground
beneath their paws is composed of fragmented glass and glowing moss, forming a
tessellated path through a gravity-defying jungle of floating vines and fractured
marble ruins. Towering, translucent trees grow upside-down from floating boulders,
their roots dripping stardust. The tigers’ stripes shimmer with embedded galaxies,
each step releasing bursts of light and cosmic particles. One tiger is seen leaping
through a fractured mirror, emerging on the other side as a reflection of itself
made of liquid silver. In the sky above, enormous jellyfish made of paper lanterns
drift slowly, casting kaleidoscopic shadows. Infuse the scene with vibrant tones
of emerald, sapphire, and iridescent gold to evoke a mood of mysticism, power,
and awe.
"""

# pipe = FluxPipeline.from_pretrained(
#     "black-forest-labs/FLUX.1-schnell",
#     torch_dtype=torch.float16  # Use float16 to reduce memory usage

# Use CPU Instead of GPU (Slower but Works)
pipe = FluxPipeline.from_pretrained(
    "black-forest-labs/FLUX.1-schnell"
).to("cpu")


# Generate the image using the GPU
image = pipe(
    prompt,
    guidance_scale=0.0,               # No classifier-free guidance (use 0.0 for pure sampling)
    num_inference_steps=4,            # Number of denoising steps—4 is very low (faster, lower quality)
    max_sequence_length=256,          # Maximum token length of the prompt
    generator=generator               # Use the seeded random generator
).images[0]

# Save & 📺 the Image
image.save("surreal.png")

# Display the image inline
display(image)
